# 22.2. Параметризации для REST API тестов
Для начала возьмём тест, который получает список питомцев, и попробуем поставить его на рельсы параметризации. Для этого обратимся к интеллект-карте, которую мы составляли, и вспомним, какие тесты мы проектировали для этого метода для параметра filter:

пустая строка;
my_pets;
строка = 255 символов;
строка длиной > 1000 символов;
кодировка (кириллица, китайские символы);
спецсимволы;
число.

In [ ]:
@pytest.fixture(autouse=True)
def get_api_key():
   """ Проверяем, что запрос api-ключа возвращает статус 200 и в результате содержится слово key"""

   # Отправляем запрос и сохраняем полученный ответ с кодом статуса в status, а текст ответа в result
   status, pytest.key = pf.get_api_key(valid_email, valid_password)

   # Сверяем полученные данные с нашими ожиданиями
   assert status == 200
   assert 'key' in pytest.key

   yield

   # Проверяем что статус ответа = 200 и имя питомца соответствует заданному
   assert pytest.status == 200


@pytest.mark.parametrize("filter", ['', 'my_pets'], ids= ['empty string', 'only my pets'])
def test_get_all_pets_with_valid_key(filter):
   """ Проверяем, что запрос всех питомцев возвращает не пустой список.
   Для этого сначала получаем api-ключ и сохраняем в переменную auth_key. Далее, используя этот ключ,
   запрашиваем список всех питомцев и проверяем, что список не пустой.
   Доступное значение параметра filter - 'my_pets' либо '' """

   pytest.status, result = pf.get_list_of_pets(pytest.key, filter)

   assert len(result['pets']) > 0


Напишем вспомогательную функцию, которая будет генерировать нам строку длиной в n символов в файле теста. Напишем функции, которые возвращают кириллический и китайский алфавиты в виде строки

In [ ]:
def generate_string(n):
   return "x" * n

def russian_chars():
   return 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

# Здесь мы взяли 20 популярных китайских иероглифов
def chinese_chars():
   return '的一是不了人我在有他这为之大来以个中上们'

def special_chars():
   return '|\\/!@#$%^&*()-_=+`~?"№;:[]{}'


@pytest.mark.parametrize("filter",
                        [''
                            , 'my_pets'
                            , generate_string(255)
                            , generate_string(1001)
                            , russian_chars()
                            , russian_chars().upper()
                            , chinese_chars()
                            , special_chars()
                            , 123
                         ]
   , ids=['empty string'
          , 'only my pets'
          , '255 symbols'
          , 'more than 1000 symbols'
          , 'russian'
          , 'RUSSIAN'
          , 'chinese'
          , 'specials'
          , 'digit'])
def test_get_all_pets_with_valid_key(filter):

все эти тесты при запуске не проходят. Происходит это потому, что сервер принимает только два возможных параметра filter, в остальных случаях должна возвращаться ошибка. Но ошибка не серверная, а клиентская, то есть 4хх. В данной ситуации мы можем поступить двумя способами — вынести логику проверки ответа из фикстуры tearDown в тест и разделить наш тест на два: один параметризованный тест для позитивных проверок и один для негативных.

In [ ]:
def generate_string(num):
   return "x" * num


def russian_chars():
   return 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'


def chinese_chars():
   return '的一是不了人我在有他这为之大来以个中上们'


def special_chars():
   return '|\\/!@#$%^&*()-_=+`~?"№;:[]{}'


@pytest.fixture(autouse=True)
def ket_api_key():
   #""" Проверяем, что запрос api-ключа возвращает статус 200 и в результате содержится слово key"""

   # Отправляем запрос и сохраняем полученный ответ с кодом статуса в status, а текст ответа в result
   status, pytest.key = pf.get_api_key(valid_email, valid_password)

   # Сверяем полученные данные с нашими ожиданиями
   assert status == 200
   assert 'key' in pytest.key

   yield


@pytest.mark.parametrize("filter",
                        [
                            generate_string(255)
                            , generate_string(1001)
                            , russian_chars()
                            , russian_chars().upper()
                            , chinese_chars()
                            , special_chars()
                            , 123
                        ],
                        ids =
                        [
                            '255 symbols'
                            , 'more than 1000 symbols'
                            , 'russian'
                            , 'RUSSIAN'
                            , 'chinese'
                            , 'specials'
                            , 'digit'
                        ])
def test_get_all_pets_with_negative_filter(filter):
   pytest.status, result = pf.get_list_of_pets(pytest.key, filter)

   # Проверяем статус ответа
   assert pytest.status == 400


@pytest.mark.parametrize("filter",
                        ['', 'my_pets'],
                        ids=['empty string', 'only my pets'])
def test_get_all_pets_with_valid_key(filter):
   pytest.status, result = pf.get_list_of_pets(pytest.key, filter)

   # Проверяем статус ответа
   assert pytest.status == 200
   assert len(result['pets']) > 0